In [170]:
import pandas as pd
import os, re, sys
import operator

In [171]:
class CtlFile:
    ctlFileName = ""
    lineStartPos = 1
    fieldDic = dict()
    posDic = dict()

    def __init__(self, fileName):
        self.ctlFileName = fileName
        self.parse()
    def parse(self):
        fieldPosition = 0
        ctlre = r"FIELDS TERMINATED BY ''.\((.*)\)"
        isFirstField = True
        text = file(self.ctlFileName).read()

        ctlreobj = re.findall(ctlre, text, re.MULTILINE | re.DOTALL)
        fieldsText = ctlreobj[0].lstrip().rstrip()

        for field in fieldsText.split(','):
            if isFirstField == True:
                x = re.findall("position\((\d+)\)", field)
                if len(x) > 0:
                    self.lineStartPos = int(x[0])
                isFirstField = False
            fname = field.replace('\t', ' ').lstrip().rstrip().split(' ')[0].lstrip().rstrip()
            if fname.upper() not in ['PLAN_ID', 'LAST_UPDATE_LOGIN', 'LAST_UPDATE_DATE', 'CREATION_DATE', 'LOW_LEVEL_CODE', 'LAST_REPLAN_DATE']:
                self.fieldDic[fname] = fieldPosition
                self.posDic[fieldPosition + 1] = fname
            fieldPosition = fieldPosition + 1
            
    def printMsg(self):
        print self.fieldDic
        

In [172]:
def get_columns(ctl_file_name):
    ctl_fp = CtlFile(ctl_file_name)
    sorted_x = sorted(ctl_fp.fieldDic.items(), key=operator.itemgetter(1))
    columns = map(lambda (k,v): k, sorted_x)
    return columns

fp_columns = get_columns("D:\\projects\\scripts_to_pbn\\test_case\\mbpoutput_good\\MSLD_FULL_PEGGING.ctl.x");

fp_columns = ["insert"]
fp_columns += get_columns("D:\\projects\\scripts_to_pbn\\test_case\\mbpoutput_good\\MSLD_FULL_PEGGING.ctl.x");
fp_columns +=["dummy1","dummy2","dummy3"]
print fp_columns, len(fp_columns)

['insert', 'pegging_id', 'inventory_item_id', 'organization_id', 'sr_instance_id', 'demand_id', 'end_pegging_id', 'demand_quantity', 'transaction_id', 'supply_type', 'supply_quantity', 'allocated_quantity', 'prev_pegging_id', 'end_item_usage', 'supply_date', 'demand_date', 'disposition_id', 'status', 'updated', 'last_updated_by', 'created_by', 'project_id', 'task_id', 'unit_number', 'end_origination_type', 'reserved_qty', 'dummy1', 'dummy2', 'dummy3'] 29


In [173]:
def load_shema(schema_file_path):
    schema = {}
    print "Load schema from: " + schema_file_path
    col = 0
    with open(schema_file_path) as schema_file:
        for line in schema_file:
            col = line.strip().split()
            if len(col) > 1:
                try:
                    row_num = int(col[0])
                except:
                    print 'ERROR: row_num ' + col[0] + ' is not an integer number'
                    sys.exit(1)
                schema[row_num] = col[len(col)-1]
    return schema

In [174]:
data_delimiter = "\022"
cn_folder = "ColumnNames"
#cn_path = os.path.join(".", cn_folder)
cn_path = "D:\\projects\\scripts_to_pbn\\ColumnNames"
full_pegging_shema_path = os.path.join(cn_path, "MSLD_FULL_PEGGING.dat")
supplies_shema_path     = os.path.join(cn_path, "MSLD_SUPPLIES.dat")
demands_shema_path      = os.path.join(cn_path, "MSLD_DEMANDS.dat")
exceptions_shema_path   = os.path.join(cn_path, "MSLD_EXCEPTION_DETAIL.dat")

#mbpoutput_path = os.path.join(".", "mbpoutput")
mbpoutput_path = "D:\\projects\\scripts_to_pbn\\test_case\\mbpoutput_bad"
#mbpoutput_path = "D:\\bugs2\\25215185\\data21\\data21\\mbpoutput"
full_pegging_ff_path = os.path.join(mbpoutput_path, "MSLD_FULL_PEGGING.dat")
supplies_ff_path     = os.path.join(mbpoutput_path, "MSLD_SUPPLIES.dat")
demands_ff_path      = os.path.join(mbpoutput_path, "MSLD_DEMANDS.dat")
exceptions_ff_path   = os.path.join(mbpoutput_path, "MSLD_EXCEPTION_DETAIL.dat")

full_pegging_shema = load_shema(full_pegging_shema_path)
supplies_shema = load_shema(supplies_shema_path)
demands_shema = load_shema(demands_shema_path)

Load schema from: D:\projects\scripts_to_pbn\ColumnNames\MSLD_FULL_PEGGING.dat
Load schema from: D:\projects\scripts_to_pbn\ColumnNames\MSLD_SUPPLIES.dat
Load schema from: D:\projects\scripts_to_pbn\ColumnNames\MSLD_DEMANDS.dat


In [175]:
print full_pegging_shema

{1: 'dummy_long', 2: 'id', 3: 'item_id', 4: 'org_id', 5: 'sr_instance_id', 6: 'demand_id', 7: 'end_pegging_id', 8: 'demand_qty', 9: 'trans_id', 10: 'supply_type', 11: 'supply_qty', 12: 'alloc_qty', 13: 'prev_level_id', 14: 'end_usage', 15: 'supply_date', 16: 'demand_date', 17: 'disp_id', 18: 'dummy_long', 19: 'dummy_long', 20: 'user_id', 21: 'dummy_long', 22: 'project_id', 23: 'task_id', 24: 'unit_num', 25: 'end_orig_type', 26: 'reserved_qty', 27: 'orig_end_priority', 28: 'end_priority', 29: 'peg_loc'}


In [176]:
#  Common on-hand pegs to excess, but we created planned orders for that item
def check_planned_order(check_onhand2excess):
    df_supplies = pd.read_csv(supplies_ff_path, sep=data_delimiter, names=supplies_shema.values())
    df_planned_order = df_supplies[(df_supplies.order_type == 5) & (df_supplies.firm_type == 2)]
    if df_planned_order.item_id.count() > 0:
        df_item_org_instance = df_planned_order[["item_id", "org_id", "sr_instance_id"]].drop_duplicates()
        results = check_onhand2excess.merge(df_item_org_instance, on=["item_id", "org_id", "sr_instance_id"])
        output = results[["item_id", "org_id", "sr_instance_id","trans_id", "supply_qty", "alloc_qty"]]
        if output.item_id.count() > 0:
            print "Found: " + str(output.item_id.count())
            print output
        else:
            print "DON'T find."

def comm_oh_excess_pl_ord():
    print "Checking 'Common on-hand pegs to excess, but we created planned orders for that item'..."
    df_full_pegging = pd.read_csv(full_pegging_ff_path, sep=data_delimiter, names=full_pegging_shema.values())
    check_onhand2excess = df_full_pegging[(df_full_pegging.supply_type == 18) & (df_full_pegging.demand_id == -1) & (df_full_pegging.project_id == -23453)]
    if check_onhand2excess.item_id.count() > 0:
        check_planned_order(check_onhand2excess)
    else:
        print "DON'T find."

In [177]:
# Unmet demands
def check_unmet_demands():
    command_str = "awk -F '\022' '{if($2==67) {print $1, $2,$3,$4, $8,$11}}' " + exceptions_ff_path + " | wc -l"
    print "Checking unmet demands with following commands: "
    print command_str
    rt=os.popen(command_str)
    output = rt.readlines();
    if len(output) > 0:
        print "Found " + str(output[0].strip()) + " unmet demands!"
    else:
        print "Errors in checking unmet demands"

In [181]:
'''
Supplies belonging to a certain project (PG1), were allocated to demands in of a different project/planning group (PG2). 
The reservation level on the plan is planning group.
'''

def check_cross_peg_across_pg():
    print "Checking cross pegging across planning group..."
    df_supplies = pd.read_csv(supplies_ff_path, sep=data_delimiter, names=supplies_shema.values())
    df_full_pegging = pd.read_csv(full_pegging_ff_path, sep=data_delimiter, names=full_pegging_shema.values())
    df_demands = pd.read_csv(demands_ff_path, sep=data_delimiter, names=demands_shema.values())
    
    join_output = df_full_pegging.merge(df_supplies, on=["item_id", "org_id", "sr_instance_id", "trans_id"])\
               .merge(df_demands, on=["item_id", "org_id", "sr_instance_id", "demand_id"])
    checking_output = join_output[((join_output.planning_group_x == -23453) & # ms.planning_group IS NULL
           (join_output.planning_group_y == -23453) &   # md.planning_group IS NULL
           (join_output.project_id_y != -23453) &       # ms.project_id IS NOT NULL
           (join_output.project_id != -23453) &         # md.project_id IS NOT NULL
           (join_output.project_id_y != join_output.project_id))  # ms.project_id <> md.project_id
            |
            ((join_output.planning_group_y != -23453) &   # md.planning_group IS NOT NULL
             (join_output.planning_group_x != -23453) &   # ms.planning_group IS NOT NULL     
             (join_output.planning_group_x != join_output.planning_group_y) # ms.planning_group <> md.planning_group
            )
            |
            ((join_output.planning_group_y == -23453) &   # md.planning_group IS NULL
             (join_output.planning_group_x != -23453)     # ms.planning_group IS NOT NULL
            )
            |
            ((join_output.project_id == -23453) &         # md.project_id IS NULL
             (join_output.project_id_y != -23453)         # ms.project_id IS NOT NULL
            )
           ]
    if checking_output.item_id.count() > 0:
        print "Found " + str(checking_output.item_id.count()) + " cross pegging records"
        '''print checking_output[["item_id", "org_id", "sr_instance_id", "trans_id", "supply_qty",\
                               "alloc_qty","supply_type","demand_id", "project_id_y","planning_group_x",\
                               "project_id", "planning_group_y"]]'''
    else:
        print "DON'T find any cross pegging acorss planning group"

In [182]:
comm_oh_excess_pl_ord()
check_unmet_demands()
check_cross_peg_across_pg()

Checking 'Common on-hand pegs to excess, but we created planned orders for that item'...
Found: 5746
      item_id  org_id  sr_instance_id    trans_id  supply_qty  alloc_qty
0       18071    1747               1  1755872159      4175.0     4142.0
1        3678    1747               1  1755879435        19.0        1.0
2        5677    1747               1  1755879936         2.0        2.0
3        3438    1747               1  1755868560        45.0       24.0
4        3448    1747               1  1755874813        25.0        6.0
5       18068    1778               1  1755936447       246.0      148.0
6       18071    1768               1  1755891126         3.0        3.0
7       18113    1747               1  1755872157        13.0        6.0
8       18115    1747               1  1755863959        88.0       66.0
9       18115    1747               1  1755861376         9.0        9.0
10       3117    1747               1  1755861739        32.0       19.0
11       3371    1747  